<a href="https://colab.research.google.com/github/dpredie/SingleHDR/blob/master/SingleHDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Run Prep to clone git and dependencies


In [1]:
!mkdir -p output
!mkdir -p input

!git clone https://github.com/dpredie/SingleHDR

!pip install numpy opencv-python tensorflow

Cloning into 'SingleHDR'...
remote: Enumerating objects: 1381, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 1381 (delta 24), reused 0 (delta 0), pack-reused 1321
Receiving objects: 100% (1381/1381), 175.17 MiB | 29.68 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [3]:
#prep section 2 - download Large model 400 MB! - skip if running small model 

%cd /content/SingleHDR/
!gdown https://drive.google.com/uc?id=1LXP7HFoVKIC8mBPHN1VW9JnQlQPtEfbO
#!unzip ckpt.zip

/content/SingleHDR
Downloading...
From: https://drive.google.com/uc?id=1LXP7HFoVKIC8mBPHN1VW9JnQlQPtEfbO
To: /content/SingleHDR/ckpt.zip
100% 609M/609M [00:03<00:00, 172MB/s]


2. Before running please upload your 360 image to /content/input/ folder. change the below input.png name to your file name




In [ ]:
# run
%cd /content/TCMonoDepth/

# Large Model - comment # to run small model
!python test_real.py --ckpt_path_deq ckpt_deq/model.ckpt --ckpt_path_lin ckpt_lin/model.ckpt --ckpt_path_hal ckpt_hal/model.ckpt --test_imgs ./imgs --output_path output_hdrs

# Small Model - uncomment to run
#!python test_real_refinement.py --ckpt_path ckpt_deq_lin_hal_ref/model.ckpt --test_imgs ./imgs --output_path output_hdrs